In [3]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

In [4]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [5]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [6]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [7]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [8]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [9]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [10]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c

In [11]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [12]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [19]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [12]:
# Initializing parameters

epochs = 35                   
batch_size = 32                  
n_hidden = 256                  
drop_out = 0.65

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [24]:
X_train.shape

(7352, 128, 9)

- Defining the Architecture of LSTM

## LSTM model with 1 layer

#### Example 1 

In [17]:
import warnings
warnings.filterwarnings("ignore")

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(drop_out))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

W1010 23:19:21.719681 11976 nn_ops.py:4224] Large dropout rate: 0.65 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               272384    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 273,926
Trainable params: 273,926
Non-trainable params: 0
_________________________________________________________________


In [18]:
import warnings
warnings.filterwarnings("ignore")
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

W1010 23:19:51.326107 11976 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1010 23:19:51.372969 11976 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [19]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

W1010 23:20:01.681306 11976 deprecation.py:323] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/35
7352/7352 [==============================] - 142s 19ms/step - loss: 1.3411 - acc: 0.4002 - val_loss: 1.2544 - val_acc: 0.4768
Epoch 2/35
7352/7352 [==============================] - 134s 18ms/step - loss: 1.2234 - acc: 0.4752 - val_loss: 1.5577 - val_acc: 0.3244
Epoch 3/35
7352/7352 [==============================] - 137s 19ms/step - loss: 1.1260 - acc: 0.5276 - val_loss: 1.1605 - val_acc: 0.5087
Epoch 4/35
7352/7352 [==============================] - 135s 18ms/step - loss: 1.1302 - acc: 0.5150 - val_loss: 0.8060 - val_acc: 0.6057
Epoch 5/35
7352/7352 [==============================] - 135s 18ms/step - loss: 0.7858 - acc: 0.6295 - val_loss: 0.7274 - val_acc: 0.6488
Epoch 6/35
7352/7352 [==============================] - 135s 18ms/step - loss: 0.6661 - acc: 0.7057 - val_loss: 0.6510 - val_acc: 0.7418
Epoch 7/35
7352/7352 [==============================] - 135s 18ms/step - loss: 0.4786 - acc: 0.8263 - val_loss: 0.4258 - val_acc: 

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      375       115        1                   0   
STANDING                 0       57       473        0                   0   
WALKING                  0        2         2      477                   6   
WALKING_DOWNSTAIRS       0        0         1       37                 376   
WALKING_UPSTAIRS         0        0         0       39                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           2  
WALKING                            9  
WALKING_DOWNSTAIRS                 6  
WALKING_UPSTAIRS                 432  


In [21]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 9s 3ms/step


In [22]:
score

[0.6165444772455196, 0.9060061079063454]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

#### Example 2

In [23]:
# Initializing parameters

epochs = 30                   
batch_size = 32                  
n_hidden = 512                  
drop_out = 0.80

In [24]:
import warnings
warnings.filterwarnings("ignore")

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(drop_out))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

W1011 00:51:38.137266 11976 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 512)               1069056   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 3078      
Total params: 1,072,134
Trainable params: 1,072,134
Non-trainable params: 0
_________________________________________________________________


In [25]:
import warnings
warnings.filterwarnings("ignore")
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [26]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 629s 86ms/step - loss: 1.5089 - acc: 0.3402 - val_loss: 1.3403 - val_acc: 0.4092
Epoch 2/30
7352/7352 [==============================] - 649s 88ms/step - loss: 1.5295 - acc: 0.3716 - val_loss: 1.5504 - val_acc: 0.3739
Epoch 3/30
7352/7352 [==============================] - 629s 86ms/step - loss: 1.3803 - acc: 0.4115 - val_loss: 1.3213 - val_acc: 0.4174
Epoch 4/30
7352/7352 [==============================] - 547s 74ms/step - loss: 1.1767 - acc: 0.4973 - val_loss: 0.8094 - val_acc: 0.6030
Epoch 5/30
7352/7352 [==============================] - 289s 39ms/step - loss: 0.8387 - acc: 0.6269 - val_loss: 0.7467 - val_acc: 0.6977
Epoch 6/30
7352/7352 [==============================] - 286s 39ms/step - loss: 0.7446 - acc: 0.6771 - val_loss: 0.6599 - val_acc: 0.7119
Epoch 7/30
7352/7352 [==============================] - 296s 40ms/step - loss: 0.5344 - acc: 0.8040 - val_loss: 0.5986 - val_acc: 

In [27]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                WALKING
True                       
LAYING                  537
SITTING                 491
STANDING                532
WALKING                 496
WALKING_DOWNSTAIRS      420
WALKING_UPSTAIRS        471


In [28]:
score = model.evaluate(X_test, Y_test,verbose=0)

2947/2947 [==============================] - 31s 11ms/step


In [32]:
print(score[0])
print(score[1])

nan
0.168306752629793


#### Example 3 

In [33]:
# Initializing parameters

epochs = 30                   
batch_size = 32                  
n_hidden = 128                  
drop_out = 0.50

In [34]:
import warnings
warnings.filterwarnings("ignore")

# Initiliazing the sequential model
model_1 = Sequential()
# Configuring the parameters
model_1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_1.add(Dropout(drop_out))
# Adding a dense output layer with sigmoid activation
model_1.add(Dense(n_classes, activation='sigmoid'))
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               70656     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [35]:
import warnings
warnings.filterwarnings("ignore")
# Compiling the model
model_1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [36]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model_1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 29s 4ms/step - loss: 1.3071 - acc: 0.4188 - val_loss: 1.2370 - val_acc: 0.4123
Epoch 2/30
7352/7352 [==============================] - 28s 4ms/step - loss: 1.1652 - acc: 0.4850 - val_loss: 1.1890 - val_acc: 0.4890
Epoch 3/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.9696 - acc: 0.5760 - val_loss: 0.8628 - val_acc: 0.6468
Epoch 4/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.7924 - acc: 0.6372 - val_loss: 0.8567 - val_acc: 0.5969
Epoch 5/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.6695 - acc: 0.6858 - val_loss: 0.6848 - val_acc: 0.6780
Epoch 6/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.5966 - acc: 0.7522 - val_loss: 0.5937 - val_acc: 0.8042
Epoch 7/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.3786 - acc: 0.8674 - val_loss: 0.5302 - val_acc: 0.8198
Epoch 8

In [37]:
score_1 = model_1.evaluate(X_test, Y_test,verbose=0)

In [39]:
print("Test Score " , score_1[0])
print("Test Accuracy " , score_1[1])

Test Score  0.3031002142316866
Test Accuracy  0.9260264675941635


#### Example 4 

In [15]:
# Initializing parameters

epochs = 25                   
batch_size = 128                  
n_hidden = 512                  
drop_out = 0.80

In [17]:
import warnings
warnings.filterwarnings("ignore")

# Initiliazing the sequential model
model_1 = Sequential()
# Configuring the parameters
model_1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_1.add(Dropout(drop_out))
# Adding a dense output layer with sigmoid activation
model_1.add(Dense(n_classes, activation='sigmoid'))
model_1.summary()

W1019 20:37:39.842547 13684 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 512)               1069056   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 1,072,134
Trainable params: 1,072,134
Non-trainable params: 0
_________________________________________________________________


In [18]:
import warnings
warnings.filterwarnings("ignore")
# Compiling the model
model_1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

W1019 20:37:46.794115 13684 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1019 20:37:46.840937 13684 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [19]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model_1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

W1019 20:37:58.244091 13684 deprecation.py:323] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 355s 48ms/step - loss: 1.4378 - acc: 0.3539 - val_loss: 1.5373 - val_acc: 0.3488
Epoch 2/25
7352/7352 [==============================] - 361s 49ms/step - loss: 1.3609 - acc: 0.4019 - val_loss: 1.5678 - val_acc: 0.3420
Epoch 3/25
7352/7352 [==============================] - 362s 49ms/step - loss: 1.6731 - acc: 0.2920 - val_loss: 1.4541 - val_acc: 0.3414
Epoch 4/25
7352/7352 [==============================] - 357s 49ms/step - loss: 1.4820 - acc: 0.3636 - val_loss: 1.7327 - val_acc: 0.2050
Epoch 5/25
7352/7352 [==============================] - 361s 49ms/step - loss: 1.4496 - acc: 0.4004 - val_loss: 1.4869 - val_acc: 0.3580
Epoch 6/25
7352/7352 [==============================] - 405s 55ms/step - loss: 1.2548 - acc: 0.4641 - val_loss: 1.2893 - val_acc: 0.4917
Epoch 7/25
7352/7352 [==============================] - 361s 49ms/step - loss: 1.4773 - acc: 0.3862 - val_loss: 1.7715 - val_acc: 

In [20]:
score_1 = model_1.evaluate(X_test, Y_test,verbose=0)

In [21]:
print("Test Score " , score_1[0])
print("Test Accuracy " , score_1[1])

Test Score  0.37387920911103095
Test Accuracy  0.8673227010519172


## LSTM model with 2 layers

#### Example 1 

In [43]:
epochs = 30                  
batch_size= 32
n_hidden_layer1 = 128
n_hidden_layer2 =64
drop_out_1 = 0.2
drop_out_2 = 0.5


In [44]:
from keras.layers.normalization import BatchNormalization

# Initiliazing the sequential model
model_2 = Sequential()
# Configuring the parameters
model_2.add(LSTM(n_hidden_layer1, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_2.add(Dropout(drop_out_1))
# Adding batch normalization
model_2.add(BatchNormalization())

model_2.add(LSTM(n_hidden_layer2))
# Adding a dropout layer
model_2.add(Dropout(drop_out_2))
# Adding a dense output layer with sigmoid activation
model_2.add(Dense(n_classes, activation='sigmoid'))
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128)          512       
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 390       
Total params: 120,966
Trainable params: 120,710
Non-trainable params: 256
_________________________________________________________________


In [45]:
# Compiling the model
model_2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [46]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model_2.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 110s 15ms/step - loss: 0.7840 - acc: 0.7444 - val_loss: 0.4422 - val_acc: 0.8663
Epoch 2/30
7352/7352 [==============================] - 133s 18ms/step - loss: 0.3214 - acc: 0.9034 - val_loss: 0.4218 - val_acc: 0.8537
Epoch 3/30
7352/7352 [==============================] - 132s 18ms/step - loss: 0.2339 - acc: 0.9226 - val_loss: 0.2183 - val_acc: 0.9203
Epoch 4/30
7352/7352 [==============================] - 133s 18ms/step - loss: 0.1915 - acc: 0.9319 - val_loss: 0.2653 - val_acc: 0.9053
Epoch 5/30
7352/7352 [==============================] - 136s 19ms/step - loss: 0.1769 - acc: 0.9385 - val_loss: 0.2450 - val_acc: 0.9067
Epoch 6/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.1631 - acc: 0.9380 - val_loss: 0.2529 - val_acc: 0.8982
Epoch 7/30
7352/7352 [==============================] - 145s 20ms/step - loss: 0.1516 - acc: 0.9423 - val_loss: 0.2489 - val_acc: 

In [47]:
score_2 = model_2.evaluate(X_test, Y_test,verbose=0)

In [48]:
print("Test Score " , score_2[0])
print("Test Accuracy " , score_2[1])

Test Score  0.28329760189572595
Test Accuracy  0.9243298269426535


#### Example 2 

In [49]:
epochs = 50                
batch_size= 64
n_hidden_layer1 = 32
n_hidden_layer2 =64
drop_out_1 = 0.5
drop_out_2 = 0.5

In [50]:
from keras.layers.normalization import BatchNormalization

# Initiliazing the sequential model
model_3 = Sequential()
# Configuring the parameters
model_3.add(LSTM(n_hidden_layer1, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_3.add(Dropout(drop_out_1))
# Adding batch normalization
model_3.add(BatchNormalization())

model_3.add(LSTM(n_hidden_layer2))
# Adding a dropout layer
model_3.add(Dropout(drop_out_2))
# Adding a dense output layer with sigmoid activation
model_3.add(Dense(n_classes, activation='sigmoid'))
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 32)           128       
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 390       
Total params: 30,726
Trainable params: 30,662
Non-trainable params: 64
_________________________________________________________________


In [51]:
# Compiling the model
model_3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [52]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model_3.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/50
7352/7352 [==============================] - 54s 7ms/step - loss: 1.1300 - acc: 0.5647 - val_loss: 0.9103 - val_acc: 0.6525
Epoch 2/50
7352/7352 [==============================] - 58s 8ms/step - loss: 0.7584 - acc: 0.6930 - val_loss: 0.7881 - val_acc: 0.6451
Epoch 3/50
7352/7352 [==============================] - 52s 7ms/step - loss: 0.5752 - acc: 0.7553 - val_loss: 0.6252 - val_acc: 0.7723
Epoch 4/50
7352/7352 [==============================] - 50s 7ms/step - loss: 0.4374 - acc: 0.8455 - val_loss: 0.4508 - val_acc: 0.8473
Epoch 5/50
7352/7352 [==============================] - 50s 7ms/step - loss: 0.3366 - acc: 0.8964 - val_loss: 0.4305 - val_acc: 0.8490
Epoch 6/50
7352/7352 [==============================] - 50s 7ms/step - loss: 0.2598 - acc: 0.9193 - val_loss: 0.3325 - val_acc: 0.8748
Epoch 7/50
7352/7352 [==============================] - 50s 7ms/step - loss: 0.2251 - acc: 0.9214 - val_loss: 0.3800 - val_acc: 0.8809
Epoch 8

### Using CNN 

#### Model 1 

In [22]:
# Importing libraries
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [25]:
epochs = 25                
batch_size= 128

In [26]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

W1020 11:39:41.144582  4124 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1020 11:39:41.472668  4124 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1020 11:39:41.472668  4124 deprecation.py:506] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1020 11:39:41.472668  4124 nn_ops.py:4224] Large dropout 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 126, 32)           896       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout_1 (Dropout)          (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1984)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                99250     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total para

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W1020 11:39:48.689685  4124 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1020 11:39:48.736552  4124 deprecation_wrapper.py:119] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [28]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

W1020 11:39:52.860619  4124 deprecation.py:323] From c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 8s 1ms/step - loss: 0.7660 - acc: 0.6827 - val_loss: 0.6110 - val_acc: 0.7811
Epoch 2/25
7352/7352 [==============================] - 6s 880us/step - loss: 0.3490 - acc: 0.8629 - val_loss: 0.4754 - val_acc: 0.8480
Epoch 3/25
7352/7352 [==============================] - 6s 875us/step - loss: 0.2435 - acc: 0.9063 - val_loss: 0.3927 - val_acc: 0.8670
Epoch 4/25
7352/7352 [==============================] - 6s 875us/step - loss: 0.1837 - acc: 0.9310 - val_loss: 0.3400 - val_acc: 0.8924
Epoch 5/25
7352/7352 [==============================] - 6s 884us/step - loss: 0.1522 - acc: 0.9406 - val_loss: 0.3324 - val_acc: 0.8870
Epoch 6/25
7352/7352 [==============================] - 7s 905us/step - loss: 0.1385 - acc: 0.9423 - val_loss: 0.3143 - val_acc: 0.8958
Epoch 7/25
7352/7352 [==============================] - 6s 880us/step - loss: 0.1210 - acc: 0.9501 - val_loss: 0.2878 - val_acc: 0.9111
Ep

In [29]:
score = model.evaluate(X_test, Y_test,verbose=0)

In [31]:
print("Test Score " , score[0])
print("Test Accuracy " , score[1])

Test Score  0.40516277702925346
Test Accuracy  0.8954869358669834


#### Model 2 

In [40]:
epochs = 25                
batch_size= 128

In [46]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=5, activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.8))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.8))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 124, 128)          5888      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 120, 64)           41024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 120, 64)           0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 60, 64)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 57, 64)            16448     
_________________________________________________________________
dropout_7 (Dropout)          (None, 57, 64)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3648)              0         
__________

In [47]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/25
7352/7352 [==============================] - 28s 4ms/step - loss: 1.5818 - acc: 0.3770 - val_loss: 1.0215 - val_acc: 0.6427
Epoch 2/25
7352/7352 [==============================] - 28s 4ms/step - loss: 0.6450 - acc: 0.7067 - val_loss: 0.7343 - val_acc: 0.6356
Epoch 3/25
7352/7352 [==============================] - 27s 4ms/step - loss: 0.4303 - acc: 0.8186 - val_loss: 0.5463 - val_acc: 0.8127
Epoch 4/25
7352/7352 [==============================] - 27s 4ms/step - loss: 0.2856 - acc: 0.8857 - val_loss: 0.5511 - val_acc: 0.8045
Epoch 5/25
7352/7352 [==============================] - 27s 4ms/step - loss: 0.2004 - acc: 0.9208 - val_loss: 0.4848 - val_acc: 0.8415
Epoch 6/25
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1779 - acc: 0.9301 - val_loss: 0.4217 - val_acc: 0.8856
Epoch 7/25
7352/7352 [==============================] - 28s 4ms/step - loss: 0.1595 - acc: 0.9392 - val_loss: 0.4178 - val_acc: 0.8853
Epoch 8

In [49]:
score = model.evaluate(X_test, Y_test,verbose=0)

In [50]:
print("Test Score " , score[0])
print("Test Accuracy " , score[1])

Test Score  0.3524832843419216
Test Accuracy  0.9168646080760094


#### Model 3 

In [51]:
epochs = 40                
batch_size= 128

In [52]:
model = Sequential()
model.add(Conv1D(filters=246, kernel_size=6, activation='relu',kernel_initializer='random_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu',kernel_initializer='random_uniform'))
model.add(Dropout(0.88))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=6, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.89))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 123, 246)          13530     
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 119, 128)          157568    
_________________________________________________________________
dropout_8 (Dropout)          (None, 119, 128)          0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 54, 64)            49216     
_________________________________________________________________
dropout_9 (Dropout)          (None, 54, 64)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3456)              0         
__________

In [53]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/40
7352/7352 [==============================] - 64s 9ms/step - loss: 1.1683 - acc: 0.4629 - val_loss: 0.8387 - val_acc: 0.6624
Epoch 2/40
7352/7352 [==============================] - 63s 9ms/step - loss: 0.5551 - acc: 0.7514 - val_loss: 0.6162 - val_acc: 0.7978
Epoch 3/40
7352/7352 [==============================] - 63s 9ms/step - loss: 0.2940 - acc: 0.8890 - val_loss: 0.4443 - val_acc: 0.8907
Epoch 4/40
7352/7352 [==============================] - 63s 9ms/step - loss: 0.1718 - acc: 0.9342 - val_loss: 0.4105 - val_acc: 0.8972
Epoch 5/40
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1437 - acc: 0.9421 - val_loss: 0.5445 - val_acc: 0.8833
Epoch 6/40
7352/7352 [==============================] - 63s 9ms/step - loss: 0.1345 - acc: 0.9425 - val_loss: 0.4486 - val_acc: 0.9033
Epoch 7/40
7352/7352 [==============================] - 64s 9ms/step - loss: 0.1231 - acc: 0.9490 - val_loss: 0.2757 - val_acc: 0.9175
Epoch 8

In [55]:
score = model.evaluate(X_test, Y_test,verbose=0)

In [56]:
print("Test Score " , score[0])
print("Test Accuracy " , score[1])

Test Score  0.4384836606237519
Test Accuracy  0.9216152019002375


#### Model 4 

In [57]:
epochs = 30                
batch_size= 128

In [59]:
model = Sequential()
model.add(Conv1D(filters=512, kernel_size=6, activation='relu',kernel_initializer='random_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=256, kernel_size=5, activation='relu',kernel_initializer='random_uniform'))
model.add(Dropout(0.88))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=6, activation='relu',kernel_initializer='random_uniform'))
model.add(Dropout(0.9))
model.add(Conv1D(filters=128, kernel_size=6, activation='relu',kernel_initializer='random_uniform'))
model.add(Dropout(0.9))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_20 (Conv1D)           (None, 123, 512)          28160     
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 119, 256)          655616    
_________________________________________________________________
dropout_12 (Dropout)         (None, 119, 256)          0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 59, 256)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 54, 128)           196736    
_________________________________________________________________
dropout_13 (Dropout)         (None, 54, 128)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 49, 128)           98432     
__________

In [60]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
start = datetime.now()

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

print("Time taken : ", datetime.now() - start)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 215s 29ms/step - loss: 12.0574 - acc: 0.1881 - val_loss: 13.1811 - val_acc: 0.1822
Epoch 2/30
7352/7352 [==============================] - 213s 29ms/step - loss: 13.0335 - acc: 0.1914 - val_loss: 13.1811 - val_acc: 0.1822
Epoch 3/30
7352/7352 [==============================] - 213s 29ms/step - loss: 13.0240 - acc: 0.1919 - val_loss: 13.1811 - val_acc: 0.1822
Epoch 4/30
7352/7352 [==============================] - 213s 29ms/step - loss: 13.0335 - acc: 0.1914 - val_loss: 13.1811 - val_acc: 0.1822
Epoch 5/30
7352/7352 [==============================] - 213s 29ms/step - loss: 13.0335 - acc: 0.1914 - val_loss: 13.1811 - val_acc: 0.1822
Epoch 6/30
7352/7352 [==============================] - 217s 29ms/step - loss: 13.0335 - acc: 0.1914 - val_loss: 13.1811 - val_acc: 0.1822
Epoch 7/30
7352/7352 [==============================] - 222s 30ms/step - loss: 13.0335 - acc: 0.1914 - val_loss: 13.18

In [62]:
score = model.evaluate(X_test, Y_test,verbose=0)

In [63]:
print("Test Score " , score[0])
print("Test Accuracy " , score[1])

Test Score  13.181068860517748
Test Accuracy  0.18221920597217509


### Prettytable for layer 1

In [66]:
number       = [1, 2, 3, 4]
epochs       = [35, 30, 30,25]
batch_size   = [32, 32, 32,128]
n_hidden     = [256, 512, 128,512]
drop_out     = [0.65, 0.80, 0.50,0.80]
accuracy     = [90.60, 16.84, 92.60, 86.73]

# Initializing prettytable 
ptable = PrettyTable()
ptable.add_column("Example",number)
ptable.add_column("Epochs", epochs)
ptable.add_column("Batch Size",batch_size)
ptable.add_column("Hidden Layer",n_hidden) 
ptable.add_column("Dropout",drop_out) 
ptable.add_column("Accuracy %",accuracy) 
print(ptable)


+---------+--------+------------+--------------+---------+------------+
| Example | Epochs | Batch Size | Hidden Layer | Dropout | Accuracy % |
+---------+--------+------------+--------------+---------+------------+
|    1    |   35   |     32     |     256      |   0.65  |    90.6    |
|    2    |   30   |     32     |     512      |   0.8   |   16.84    |
|    3    |   30   |     32     |     128      |   0.5   |    92.6    |
|    4    |   25   |    128     |     512      |   0.8   |   86.73    |
+---------+--------+------------+--------------+---------+------------+


### Prettytable for layer 2 

In [58]:
number          = [1, 2]
epochs          = [30, 50]
batch_size      = [32, 64]
n_hidden_layer1 = [128, 32]
n_hidden_layer2 = [64, 64]
drop_out_1      = [0.2, 0.5]
drop_out_2      = [0.5, 0.5]
accuracy     = [92.43, 91.52]

# Initializing prettytable 
ptable = PrettyTable()
ptable.add_column("Example",number)
ptable.add_column("Epochs", epochs)
ptable.add_column("Batch Size",batch_size)
ptable.add_column("Hidden Layer 1",n_hidden_layer1)
ptable.add_column("Hidden Layer 2",n_hidden_layer2)
ptable.add_column("Dropout 1",drop_out_1)
ptable.add_column("Dropout 2",drop_out_2)
ptable.add_column("Accuracy %",accuracy) 
print(ptable)


+---------+--------+------------+----------------+----------------+-----------+-----------+------------+
| Example | Epochs | Batch Size | Hidden Layer 1 | Hidden Layer 2 | Dropout 1 | Dropout 2 | Accuracy % |
+---------+--------+------------+----------------+----------------+-----------+-----------+------------+
|    1    |   30   |     32     |      128       |       64       |    0.2    |    0.5    |   92.43    |
|    2    |   50   |     64     |       32       |       64       |    0.5    |    0.5    |   91.52    |
+---------+--------+------------+----------------+----------------+-----------+-----------+------------+


### Prettytable for CNN

In [65]:
number          = [1, 2, 3, 4]
epochs          = [25, 25, 40, 30]
batch_size      = [128, 128, 128, 128]
accuracy     = [89.54,91.68,92.16,18.22]

# Initializing prettytable 
ptable = PrettyTable()
ptable.add_column("Model",number)
ptable.add_column("Epochs", epochs)
ptable.add_column("Batch Size",batch_size)
ptable.add_column("Accuracy %",accuracy) 
print(ptable)


+-------+--------+------------+------------+
| Model | Epochs | Batch Size | Accuracy % |
+-------+--------+------------+------------+
|   1   |   25   |    128     |   89.54    |
|   2   |   25   |    128     |   91.68    |
|   3   |   40   |    128     |   92.16    |
|   4   |   30   |    128     |   18.22    |
+-------+--------+------------+------------+


### Conclusions 

1. We have used one LSTM layered model and two LSTM layered model.
2. Used different epochs, batch sizes, dropout layers to find accuracy.
3. Maximum accuracy % in one LSTM layer model is 92.60%
4. We also used larger LSTM units with large dropout rates in one layered model.
4. Accuracy % in two LSTM layers model is 92.43%
5. Earlystopping technique can help to get maximum accuracy of 93.72% in two layered architecture.
6. We have tried all different combinations of epochs, batch sizes and dropout values to find maximum accuracy
   and approx 93% is best accuracy.
7. With different CNN models, we get best accuracy of 92.16%. We have used different architectures of Conv2D with 
   different kernel sizes.